In [1]:
import os
import sys
import django
from django.db import connection
# from tqdm import tqdm
import logging
import pandas as pd
from datetime import datetime

sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")

In [12]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger('User similarity calculator')

# Import Models
from title.models import Book
from transaction.models import Master, Detail
from main_site.models import Rating
from django.db.models import Avg

In [3]:
def cal_point(sample):
    hire_date_length = abs(sample.return_date - sample.hire_date)
    due_date_length = abs(sample.due_date - sample.hire_date)
    ratio = hire_date_length / due_date_length
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio, 3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1 - ratio) / (ratio - 1))
        sample['point'] = 3 + (7 * ratio)
    sample['point'] = round(sample['point'], 2)
    return sample

In [4]:
book_df = pd.DataFrame(list(Book.objects.all().values()))
# Change transaction_master_db to data frame
transaction_master_df = pd.DataFrame(list(Master.objects.all().values()))
# Change name column
transaction_master_df.rename(columns={'date': 'hire_date'},
                             inplace=True)
# Change transaction_detail_db to data frame
transaction_detail_df = pd.DataFrame(list(Detail.objects.all().values()))
# Join two data frame to one
transaction_df = pd.merge(transaction_master_df, transaction_detail_df, left_on='id', right_on='transaction_id')
transaction_df = pd.merge(transaction_df, book_df[['barcode', 'title_id']], left_on="book_id",
                          right_on="barcode",
                          how="left")
transaction_df = transaction_df.drop(['id_x', 'transaction_id'], axis=1)
transaction_df.describe(include = "all")

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
count,198,198.000000,198,198,198.000000,198.000000,198.000000,184,198.000000,198,198.000000
unique,104,NaN,130,70,NaN,NaN,NaN,97,NaN,130,NaN
top,2017-09-14 00:00:00,NaN,52076-130,2018-02-15 00:00:00,NaN,NaN,NaN,2017-09-21 00:00:00,NaN,52076-130,NaN
freq,11,NaN,5,41,NaN,NaN,NaN,9,NaN,5,NaN
first,2017-02-15 00:00:00,NaN,NaN,2017-06-13 00:00:00,NaN,NaN,NaN,2017-06-12 00:00:00,NaN,NaN,NaN
last,2019-07-01 00:00:00,NaN,NaN,2019-08-31 00:00:00,NaN,NaN,NaN,2019-07-03 00:00:00,NaN,NaN,NaN
mean,NaN,61.727273,NaN,NaN,1.555556,99.500000,45316.628788,NaN,1.141414,NaN,76.277778
std,NaN,35.113397,NaN,NaN,0.537379,57.301832,30681.243568,NaN,0.513969,NaN,45.165444
min,NaN,5.000000,NaN,NaN,1.000000,1.000000,0.000000,NaN,1.000000,NaN,1.000000
25%,NaN,33.250000,NaN,NaN,1.000000,50.250000,17776.000000,NaN,1.000000,NaN,31.750000


In [5]:
transaction_df[transaction_df.return_date < transaction_df.hire_date]

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id


In [6]:
transaction_df[pd.isna(transaction_df.return_date)]

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
21,2017-09-14,44,53398-60,2018-02-15,2,22,100000.00,NaT,3,53398-60,97
52,2017-09-22,26,53664-90,2018-02-15,2,53,64337.00,NaT,3,53664-90,14
62,2017-09-26,55,53400-100,2018-02-15,2,63,59489.00,NaT,3,53400-100,130
157,2018-11-23,93,53627-60,2019-02-15,2,158,53025.00,NaT,3,53627-60,121
159,2019-01-11,51,53698-60,2019-01-18,1,160,11716.00,NaT,3,53698-60,12
171,2019-01-31,122,53634-90,2019-02-14,3,172,0.00,NaT,3,53634-90,116
177,2019-02-18,42,53698-190,2019-02-25,1,178,11716.00,NaT,3,53698-190,12
178,2019-02-21,46,53613-60,2019-02-28,1,179,13054.25,NaT,3,53613-60,106
182,2019-02-28,66,52076-130,2019-07-15,2,183,66256.00,NaT,3,52076-130,91
184,2019-03-06,66,53388-90,2019-07-15,2,185,55146.00,NaT,3,53388-90,101


In [7]:
transaction_df['return_date'] = transaction_df['return_date'].fillna(datetime.now())

In [8]:
transaction_df.describe(include = 'all')

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
count,198,198.000000,198,198,198.000000,198.000000,198.000000,198,198.000000,198,198.000000
unique,104,NaN,130,70,NaN,NaN,NaN,98,NaN,130,NaN
top,2017-09-14 00:00:00,NaN,52076-130,2018-02-15 00:00:00,NaN,NaN,NaN,2019-07-22 14:35:36.425831,NaN,52076-130,NaN
freq,11,NaN,5,41,NaN,NaN,NaN,14,NaN,5,NaN
first,2017-02-15 00:00:00,NaN,NaN,2017-06-13 00:00:00,NaN,NaN,NaN,2017-06-12 00:00:00,NaN,NaN,NaN
last,2019-07-01 00:00:00,NaN,NaN,2019-08-31 00:00:00,NaN,NaN,NaN,2019-07-22 14:35:36.425831,NaN,NaN,NaN
mean,NaN,61.727273,NaN,NaN,1.555556,99.500000,45316.628788,NaN,1.141414,NaN,76.277778
std,NaN,35.113397,NaN,NaN,0.537379,57.301832,30681.243568,NaN,0.513969,NaN,45.165444
min,NaN,5.000000,NaN,NaN,1.000000,1.000000,0.000000,NaN,1.000000,NaN,1.000000
25%,NaN,33.250000,NaN,NaN,1.000000,50.250000,17776.000000,NaN,1.000000,NaN,31.750000


In [9]:
transaction_df.head()

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
0,2017-02-15,38,53683-70,2017-07-15,2,1,59792.0,2018-03-19,1,53683-70,23
1,2017-02-15,40,53398-70,2017-07-15,2,2,100000.0,2017-07-17,1,53398-70,97
2,2017-06-06,120,53683-60,2017-06-13,1,3,14948.0,2017-06-12,1,53683-60,23
3,2017-07-01,68,58584-80,2017-08-31,2,4,73124.0,2018-03-19,1,58584-80,88
4,2017-06-19,26,53664-90,2017-08-31,2,5,64337.0,2017-09-22,1,53664-90,14


In [10]:
transaction_df = transaction_df.groupby('id_y').apply(cal_point).reset_index(drop=True)
transaction_df[transaction_df.return_date < transaction_df.hire_date]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:887: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis)


,barcode,book_id,due_date,due_date_length,hire_date,hire_date_length,hire_type,id_y,point,price,ratio_hire_date,return_date,status,test,title_id,user_id


In [11]:
# New data frame with user_id, title_id, point
user_rating = pd.DataFrame()
user_rating['user_id'] = transaction_df.user_id
user_rating['title_id'] = transaction_df.title_id
user_rating['point'] = transaction_df.point
user_rating_df = user_rating.groupby(['user_id', 'title_id']).mean().round(3).reset_index()
user_rating_df.head()

,user_id,title_id,point
0,5,106,8.430
1,6,21,8.440
2,6,50,3.195
3,6,106,8.430
4,7,106,8.430


In [32]:
title_ids_list = Rating.objects.filter(type='calculate').values('title_id') \
    .annotate(mean_rating=Avg('rating')) \
    .order_by('title_id')
title_ids_list_df = pd.DataFrame.from_records(title_ids_list)
print(title_ids_list.query)

SELECT `rating`.`title_id`, AVG(`rating`.`rating`) AS `mean_rating` FROM `rating` WHERE `rating`.`type` = calculate GROUP BY `rating`.`title_id` ORDER BY `rating`.`title_id` ASC


In [29]:
title_ids_list_df

,mean_rating,title_id
0,3.000000,1
1,3.000000,2
2,3.000000,6
3,3.800000,7
4,6.801667,12
5,6.946667,14
6,8.920000,15
7,5.532500,16
8,3.000000,18
9,6.975000,20
